In [41]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("sonalshinde123/student-placement-dataset")

print("Path to dataset files:", path)
print(os.listdir(path))

Path to dataset files: /Users/cjz/.cache/kagglehub/datasets/sonalshinde123/student-placement-dataset/versions/1
['Student Placement Dataset']


In [102]:
import pandas as pd

csv_files = [
    os.path.join(path, "Student Placement Dataset", "train.csv"),
    os.path.join(path, "Student Placement Dataset", "test.csv")
]

df = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)

df["Gender"] = df["Gender"].map({"Female": 0, "Male": 1})
df["Placement_Status"] = df["Placement_Status"].map({"Not Placed": 0, "Placed": 1})

columns = ["Degree", "Branch"]
df = pd.get_dummies(df, columns=["Degree", "Branch"], dtype=int) # one-hot encoding categorical cols

df = df.drop(columns=["Student_ID"])